In [1]:
import joblib
import tarfile
import os

# Package model as tar.gz
with tarfile.open("best_model.tar.gz", "w:gz") as tar:
    tar.add("pkl_joblib_files/best_model.pkl")


In [2]:
import boto3

bucket = "lead-classification-bucket"  # create a bucket if needed
model_key = "lead_conversion_model/betst_model.tar.gz"
model_uri = f"s3://{bucket}/{model_key}"

s3 = boto3.client("s3")
s3.upload_file("best_model.tar.gz", bucket, model_key)


In [11]:
from sagemaker.sklearn.model import SKLearnModel
import sagemaker

role = 'arn:aws:iam::258399971007:role/sagemaker-role'
sagemaker_session = sagemaker.Session()

sk_model = SKLearnModel(
    model_data='s3://lead-classification-bucket/model_bundle.tar.gz',
    role=role,
    entry_point='inference.py',
    framework_version='1.2-1',  # or whatever you used
    py_version='py3',
    sagemaker_session=sagemaker_session
)

predictor = sk_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name='lead-conversion-endpoint'
)

----------------------------------------------*

Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-endpoint


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:16                                                                                   │
│                                                                                                  │
│   13 │   sagemaker_session=sagemaker_session                                                     │
│   14 )                                                                                           │
│   15                                                                                             │
│ ❱ 16 predictor = sk_model.deploy(                                                                │
│   17 │   initial_instance_count=1,                                                               │
│   18 │   instance_type='ml.m5.large',                                                            │
│   19 │   endpoint_name='lead-conversion-endpoint'                                                │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/model.py:1814 in deploy                        │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/session.py:6033 in                             │
│ endpoint_from_production_variants                                                                │
│                                                                                                  │
│   6030 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6031 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6032 │   │                                                                                     │
│ ❱ 6033 │   │   return self.create_endpoint(                                                      │
│   6034 │   │   │   endpoint_name=name,                                                           │
│   6035 │   │   │   config_name=name,                                                             │
│   6036 │   │   │   tags=endpoint_tags,                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/session.py:4878 in create_endpoint             │
│                                                                                                  │
│   4875 │   │   │   logger.error(                                                                 │
│   4876 │   │   │   │   "Please check the troubleshooting guide for common errors: %s", troubles  │
│   4877 │   │   │   )                                                                             │
│ ❱ 4878 │   │   │   raise e                                                                       │
│   4879 │                                                                                         │
│   4880 │   def endpoint_in_service_or_not(self, endpoint_na

In [3]:
import boto3

sm = boto3.client('sagemaker')
sm.delete_endpoint_config(EndpointConfigName='lead-conversion-endpoint')


{'ResponseMetadata': {'RequestId': '2c2b76f3-c5fa-437d-b4ca-e9cecdbe0334',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2c2b76f3-c5fa-437d-b4ca-e9cecdbe0334',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 20 Jul 2025 07:47:49 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [5]:
import boto3

sagemaker_client = boto3.client("sagemaker", region_name="ap-south-1")

endpoint_name = "lead-conversion-endpoint"

response = sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
print("✅ Deleted endpoint:", endpoint_name)


✅ Deleted endpoint: lead-conversion-endpoint
